In [83]:
# import wandb

# wandb.login()

In [84]:
# import wandb
# import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="my-awesome-project",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

# # simulate training
# epochs = 10
# offset = random.random() / 5
# for epoch in range(2, epochs):
#     acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#     loss = 2 ** -epoch + random.random() / epoch + offset
    
#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

In [85]:
import torch
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    print("MPS is available on this machine - enabling MPS for PyTorch tensors.")

MPS not available because the current PyTorch install was not built with MPS enabled.


In [86]:
import argparse
import glob
import logging
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from transformers import (
    WEIGHTS_NAME,
    AdamW,
    PreTrainedModel,
    PreTrainedTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)

import time
import sys
import os
import wandb
import random

import torch
torch.cuda.empty_cache()

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device, n_gpu)
if n_gpu < 1:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    n_gpu = 1
    print(device, n_gpu)

cuda 2


In [87]:
import logging
# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

local_rank = -1
fp16 = False
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    local_rank,
    device,
    n_gpu,
    bool(local_rank != -1),
    fp16,
)

01/04/2024 23:28:14 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 2, distributed training: False, 16-bits training: False


In [88]:
import os
# DATASET_JAVA_TOKEN_RAW = '/home/myoungkyu@unomaha.edu/Documents/\
# 2021MSR-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/'

DATASET_JAVA_TOKEN_RAW = os.environ['HOME']+'/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/'
if os.path.isdir(DATASET_JAVA_TOKEN_RAW):
    print('[DBG] Data:', DATASET_JAVA_TOKEN_RAW)

class Args:
    def __init__(self):
        self.model_type = 'roberta'
        self.line_by_line = True
        self.model_name_or_path = ''
        self.mlm = True
        self.mlm_probability = 0.2
        self.tokenizer_name = 'roberta-base'#tokenizer_name
        self.cache_dir = 'cache'
        self.block_size = -1
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = True
        self.per_gpu_train_batch_size = 2 #4 #8 #wandb.config.batch_size
        self.gradient_accumulation_steps = 4 #wandb.config.gradient_accumulation_steps
        self.learning_rate = 5e-5 #wandb.config.learning_rate
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 1 # 50 # wandb.config.epochs
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 500 #logging_steps
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.seed = 42
        self.fp16 = False
        self.fp16_opt_level = "O1"
        self.local_rank = -1
        self.server_ip = ""
        self.server_port = ""

args = Args(
    # run, parsed_args.train_data_file, parsed_args.eval_data_file,
    # parsed_args.output_root, parsed_args.tokenizer_name, parsed_args.vocab_size,
    # parsed_args.early_stop, parsed_args.logging_steps, parsed_args.should_continue
)
args.device = device
args.n_gpu = n_gpu
args.vocab_size = 50_000
args.train_data_file = DATASET_JAVA_TOKEN_RAW + 'training_masked_code-small20per' # 20percent small
args.eval_data_file = DATASET_JAVA_TOKEN_RAW + 'eval_masked_code-small20per' # 20percent small

[DBG] Data: /home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/


In [89]:
import numpy as np

seed = 42
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

set_seed(args)

In [90]:
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    PreTrainedModel,
    PreTrainedTokenizer,
    RobertaConfig,
    RobertaForMaskedLM,
    RobertaTokenizer,
    get_linear_schedule_with_warmup,
)
MODEL_CLASSES = {
    "roberta": (RobertaConfig, RobertaForMaskedLM, RobertaTokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config_class, model_class, tokenizer_class

(transformers.models.roberta.configuration_roberta.RobertaConfig,
 transformers.models.roberta.modeling_roberta.RobertaForMaskedLM,
 transformers.models.roberta.tokenization_roberta.RobertaTokenizer)

In [91]:
import os
import glob

output_trained_tokenizer = os.path.abspath(DATASET_JAVA_TOKEN_RAW + 'tokenizer/output')
files = glob.glob(os.path.join(output_trained_tokenizer, '*'))
for file in files:
    try:
        if os.path.isfile(file):
            os.remove(file)
            print('[DBG] Removed', file)
    except Exception as e:
        print(f"Error while deleting {file}: {e}")

[DBG] Removed /home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output/vocab.json
[DBG] Removed /home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output/merges.txt


In [92]:
from tokenizers import ByteLevelBPETokenizer
import os

output_trained_tokenizer = os.path.abspath(DATASET_JAVA_TOKEN_RAW + 'tokenizer/output')

# ###################################################################################
# Note!! The following code should be executed once to save the result on the disk. 
# ###################################################################################

tokenizer = ByteLevelBPETokenizer()
train_data_file_tokenizer = DATASET_JAVA_TOKEN_RAW + 'tokenizer_training'
path = os.path.abspath(train_data_file_tokenizer)
tokenizer.train(files=path, vocab_size=50000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.add_special_tokens(["<x>","<z>"])
# Save files to disk
if not os.path.exists(output_trained_tokenizer):
    os.makedirs(output_trained_tokenizer)
saved_files = tokenizer.save_model(output_trained_tokenizer)
print(str(output_trained_tokenizer))
for f in saved_files:
    print(f)




/home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output
/home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output/vocab.json
/home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output/merges.txt


In [93]:
def get_config(args):
    config = {
        "model_type": "roberta",
        "attention_probs_dropout_prob": 0.1,
        "hidden_act": "gelu",
        "hidden_dropout_prob": 0.3,
        "hidden_size": 768, #wandb.config.hidden_size,
        "initializer_range": 0.02,
        "num_attention_heads": 16, #wandb.config.num_attention_heads,
        "num_hidden_layers": 12, #wandb.config.num_hidden_layers,
        "vocab_size": 1_130, #args.vocab_size,
        "intermediate_size": 4_096, #wandb.config.intermediate_size,
        "max_position_embeddings": 1024,
        "cache_dir": '' #args.cache_dir
    }
    return RobertaConfig(**config)

config = get_config(args)
tokenizer = tokenizer_class.from_pretrained(output_trained_tokenizer, cache_dir=args.cache_dir)
args.block_size = tokenizer.model_max_length
print(tokenizer)
print(tokenizer.model_max_length)

logger.info("Training new model from scratch")
try:
    model = model_class(config=config)
except Exception as e:
    logger.error(f'{e} Configuration not correct')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
01/04/2024 23:28:54 - INFO - __main__ -   Training new model from scratch


RobertaTokenizer(name_or_path='/home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output', vocab_size=50000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=

In [94]:
def add_special_tokens_(model, tokenizer):
    """ Add special tokens to the tokenizer and the model if they have not already been added. """
    orig_num_tokens = len(tokenizer.encoder)
    num_added_tokens = tokenizer.add_special_tokens({'additional_special_tokens': ['<z>', '<x>']}) # doesn't add if they are already there
    if num_added_tokens > 0:
        model.resize_token_embeddings(new_num_tokens=orig_num_tokens + num_added_tokens)

add_special_tokens_(model, tokenizer)
model.to(args.device)

attributes = vars(args)
for key, value in attributes.items():
    print(f"{key}: {value}")

print('[DBG] Device:', args.device)

model_type: roberta
line_by_line: True
model_name_or_path: 
mlm: True
mlm_probability: 0.2
tokenizer_name: roberta-base
cache_dir: cache
block_size: 1000000000000000019884624838656
do_train: True
do_eval: True
evaluate_during_training: True
per_gpu_train_batch_size: 2
gradient_accumulation_steps: 4
learning_rate: 5e-05
weight_decay: 0.0
adam_epsilon: 1e-08
max_grad_norm: 1.0
num_train_epochs: 1
max_steps: -1
warmup_steps: 0
logging_steps: 500
save_total_limit: None
eval_all_checkpoints: False
no_cuda: False
overwrite_output_dir: True
seed: 42
fp16: False
fp16_opt_level: O1
local_rank: -1
server_ip: 
server_port: 
device: cuda
n_gpu: 2
vocab_size: 50000
train_data_file: /home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/training_masked_code-small20per
eval_data_file: /home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/eval_masked_code-small20per
[DBG] Device: cuda


In [95]:
len(tokenizer)

50002

In [96]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from tokenizers.processors import BertProcessing

"""
In the given code snippet, `tokenizer.encode_batch(lines)` returns a list of `Encoding`
objects, each representing the tokenization of a line. The `.ids` attribute of each
`Encoding` object contains the actual token IDs produced during the tokenization process.

When you're dealing with a pretrained tokenizer, token IDs are the integer representations
of tokens in the vocabulary. For example, in a BPE tokenizer, each token is represented by
a unique integer ID.

So, `x.ids` is crucial because it gives you the actual token IDs after tokenizing the
lines using the tokenizer. These token IDs are what the model will process during
training.
"""
class LineByLineDatasetWithBPETokenizer(Dataset):
    def __init__(self, data_file_path: str = None, tokenizer_path: str = None):
        tokenizer = ByteLevelBPETokenizer(
            tokenizer_path + "/vocab.json",
            tokenizer_path + "/merges.txt",
        )
        tokenizer._tokenizer.post_processor = BertProcessing(
            ("</s>", tokenizer.token_to_id("</s>")),
            ("<s>", tokenizer.token_to_id("<s>")),
        )
        tokenizer.enable_truncation(max_length=512)

        self.examples = []

        with open(data_file_path, encoding="utf-8") as f:
            lines = f.readlines()
            lines = [line for line in lines if (len(line) > 0 and not line.isspace())]
            self.examples += [x.ids for x in tokenizer.encode_batch(lines)]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])

def load_and_cache_examples(args, evaluate=False):
    data_file_path = args.eval_data_file if evaluate else args.train_data_file
    print(data_file_path)
    args.tokenizer_name = output_trained_tokenizer
    print(args.tokenizer_name)
    return LineByLineDatasetWithBPETokenizer(data_file_path, args.tokenizer_name)

print('[DBG] output_trained_tokenizer: ', output_trained_tokenizer)
train_dataset = load_and_cache_examples(args, evaluate=False)

[DBG] output_trained_tokenizer:  /home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output
/home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/training_masked_code-small20per
/home/myoungkyu/Documents/2021MSR-StudyBert-RoBertaCode/roberta/DATASET/JAVA/TOKEN/RAW/tokenizer/output


In [97]:
counter = 0
for info in train_dataset:
    counter+=1
    if (counter < 3):
        print(info)

tensor([    0,   316,   378,  1279,  1368,  2111,  7257,    12,  7704,   731,
          630,    16,   359,   523,  8454,    13,   451,  2958,  9380,  3595,
         2012,    16,  7276,    16, 10018,   535,    92,    34,  5012,   523,
         1658,   283,   341,  5012,    63,    22, 16448,  1658,    63,    20,
           65,   283,   341,  5012,  6205,   630,   512,   391,  7704,   512,
          731,   630,  2316,  1658,    63,    21,    65,   283,   341,  5012,
          393,  8488,   512,   391,   309, 24747,  8454,  2316, 14425,   429,
         2608,   393,  1444,  1368,  2111,  7257,   512,  1658,    16,   350,
         2316,   270,   203,     2])
tensor([    0,   316,   378,  1279,  1368,  2111,  7257,    12,  7704,   731,
          630,    16,   359,   523,  8454,    13,   451,  2958,  9380,  3595,
         2012,    16,  7276,    16, 10018,    16,  2543,  1825,  1050,  5012,
          523,  1658,   535,    92,    34,  1658,    63,    20,    65,   283,
          341,  5012,  6205

In [98]:
def decision(probability):
    return random.random() < probability

In [99]:
"""
This function, `read_masked_dataset`, is designed to read a dataset of masked instances,
preprocess it, and return tensors suitable for model training. It takes a tokenizer, a
batch of masked instances, and a list of labels to process. The function decodes and
encodes these instances, pads them to the maximum length, and creates tensors for both
inputs and labels. Special tokens, such as `<s>` and `</s>`, are removed, and the labels
are padded with -100 for unmasked tokens. The resulting tensors, `inputs` and `labels`,
are then returned. Overall, the function prepares the data for masked language model
training by ensuring consistency in sequence length and appropriate tokenization.
 
The `read_masked_dataset` function processes a batch of masked instances and their
corresponding labels for model training. It utilizes a tokenizer to decode and encode
these instances, removing special tokens and ensuring a consistent length. The resulting
labels are converted into a tensor, `labels_to_convert_in_tensor`, where unmasked tokens
are padded with -100. Similarly, the masked instances are processed and converted into a
tensor, `inputs_to_convert`, with padding for a uniform length. The function returns these
two tensors, `inputs` and `labels`, providing preprocessed data suitable for training a
masked language model. # 
"""
def read_masked_dataset(tokenizer: PreTrainedTokenizer, batch, labels_to_process) -> Tuple[torch.Tensor, torch.Tensor]:
    # The inputs are already masked in the training file
    tmp_inputs = batch.clone()

    tmp_inputs_list = []
    for input in tmp_inputs:
        decoded_input = tokenizer.decode(input)
        encoded_back = tokenizer.encode(decoded_input)[1:-1] # Removes the additional <s> and </s> added
        tmp_inputs_list.append(encoded_back)

    # Gets the maximum length between inputs and labels_lines
    # We then need to adapt one or the other to have the same length through padding
    max_length_inputs = max([len(input) for input in tmp_inputs_list])
    max_length_labels_lines = max([len(label) for label in labels_to_process])
    max_length = max_length_inputs
    if max_length_labels_lines > max_length_inputs:
        max_length = max_length_labels_lines

    # Create the labels as tensor
    labels_to_convert_in_tensor = []

    i = 0
    while i < len(batch):
        l1_tmp = tokenizer.encode(labels_to_process[i])
        label_to_add = []
        for token in l1_tmp:
            if token != tokenizer.bos_token_id and token != tokenizer.eos_token_id:  # Remove special tokens
                label_to_add.append(token)

        j = len(label_to_add)
        while j < max_length:
            label_to_add.append(-100)  # we only compute loss for masked tokens
            j += 1

        labels_to_convert_in_tensor.append(label_to_add)
        i += 1

    labels = torch.as_tensor(labels_to_convert_in_tensor)

    # Create the train data set input batch as tensor.
    inputs_to_convert = []
    for input in tmp_inputs_list:
        tmp_input = []
        for token in input:
            tmp_input.append(token)

        i = len(tmp_input)
        while i < max_length:
            tmp_input.append(tokenizer.pad_token_id)
            i += 1
        inputs_to_convert.append(tmp_input)

    inputs = torch.as_tensor(inputs_to_convert)

    return inputs, labels

In [100]:
# Reads the masked instances, but provide them as non-masked to the model
# This is used in 10% of cases during training
def get_non_masked_instances(tokenizer: PreTrainedTokenizer, batch, labels_to_process) -> Tuple[torch.Tensor, torch.Tensor]:
    tmp_inputs_list = []
    i = 0
    while i < len(batch):
        decoded_input = tokenizer.decode(batch[i]).replace('<x>',str(labels_to_process[i]).replace('<z>\n',''))
        encoded_back = tokenizer.encode(decoded_input)[1:-1]
        tmp_inputs_list.append(encoded_back)
        i += 1

    # Gets the maximum length
    max_length = max([len(input) for input in tmp_inputs_list])

    inputs_to_convert = []
    labels_to_convert = []
    for input in tmp_inputs_list:
        tmp_input = []
        tmp_label = []
        tmp_label.append(tokenizer.convert_tokens_to_ids('<z>'))
        for token in input:
            tmp_input.append(token)
            tmp_label.append(-100)

        del tmp_label[-1] #Accounts for the fact that tmp_label already contains <z>

        i = len(tmp_input)
        while i < max_length:
            tmp_input.append(tokenizer.pad_token_id)
            tmp_label.append(-100)
            i += 1
        labels_to_convert.append(tmp_label)
        inputs_to_convert.append(tmp_input)

    inputs = torch.as_tensor(inputs_to_convert)
    labels = torch.as_tensor(labels_to_convert)

    # We train the model to understand that if no masked tokens are present, nothing must be produced, only <z>
    return inputs, labels

In [101]:
counter = 0
no_decay = ["bias", "LayerNorm.weight"]
for n, p in model.named_parameters():
    counter+=1
    if(counter < 10):
        print(n)
        if any(nd in n for nd in no_decay):
            print('no decay:',n)
print('Total parameters of model.named_parameters():', counter)

roberta.embeddings.word_embeddings.weight
roberta.embeddings.position_embeddings.weight
roberta.embeddings.token_type_embeddings.weight
roberta.embeddings.LayerNorm.weight
no decay: roberta.embeddings.LayerNorm.weight
roberta.embeddings.LayerNorm.bias
no decay: roberta.embeddings.LayerNorm.bias
roberta.encoder.layer.0.attention.self.query.weight
roberta.encoder.layer.0.attention.self.query.bias
no decay: roberta.encoder.layer.0.attention.self.query.bias
roberta.encoder.layer.0.attention.self.key.weight
roberta.encoder.layer.0.attention.self.key.bias
no decay: roberta.encoder.layer.0.attention.self.key.bias
Total parameters of model.named_parameters(): 202


In [103]:
from typing import Dict, List, Tuple
from torch.nn.utils.rnn import pad_sequence
from datetime import datetime

def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    t0 = datetime.now()
    # if args.local_rank in [-1, 0]:
    #     tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    # Designed to be used with PyTorch's `DataLoader`. This function takes a list of
    # tensors (examples) and uses `pad_sequence` to pad them to the same length along the
    # batch dimension. If the tokenizer has a defined padding token, it uses its ID for
    # padding; otherwise, it uses a default padding value. The result is a batch of padded
    # sequences ready for model training.
    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = SequentialSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate
    )

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    """
    This code is preparing the parameters for optimization during training. It creates a
    list of dictionaries, each corresponding to a parameter group for the optimizer. There
    are two parameter groups:

    1. **Weight Decay Group**: Parameters that should undergo weight decay. These are
       selected based on their names, and any parameters with names containing specific
       substrings (specified in `no_decay`) are excluded from weight decay. The weight
       decay for this group is set to `args.weight_decay`.

    2. **No Weight Decay Group**: Parameters that should not undergo weight decay. Again,
       parameters are selected based on their names, and any parameters with names
       containing substrings from `no_decay` are included in this group. The weight decay
       for this group is set to 0.0.

    This configuration is commonly used when fine-tuning models. Weight decay is a
    regularization term added to the loss function during training to prevent overfitting.
    However, it's often desirable not to apply weight decay to certain parameters, such as
    bias terms, which can be achieved by specifying different weight decay values for
    different parameter groups.    
    """
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
            "weight_decay": 0.0
        },
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    """ Train! """
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  args.train_batch_size: %d", args.train_batch_size)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    """ 
        Check if continuing training from a checkpoint 
        Skipped 
    """
    tr_loss, logging_loss = 0.0, 0.0
    
    model_to_resize = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model_to_resize.resize_token_embeddings(len(tokenizer))

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility

    labels_file = str(args.train_data_file).replace('masked_code_','mask_')
    labels_lines = [line.rstrip() for line in open(labels_file)]

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        # step is the count of the steps performed, batch contains the actual input data

        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            # Get the labels lines to process
            start = step * len(batch)
            end = start + len(batch) + 1
            lables_to_process = labels_lines[start:end]

            # In 90% of cases, we used the inputs with the masked tokens
            # In 10% of cases we don't mask any token
            if decision(0.9):
                inputs, labels = read_masked_dataset(tokenizer, batch, lables_to_process)
            else:
                inputs, labels = get_non_masked_instances(tokenizer, batch, lables_to_process)

            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            # outputs = model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            outputs = model(inputs, labels=labels) if args.mlm else model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            loss.backward()
            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                #     # Log metrics
                #     if (
                #         args.local_rank == -1 and args.evaluate_during_training
                #     ):  # Only evaluate when single GPU otherwise metrics may not average well
                #         results = evaluate(args, model, tokenizer)
                #         if results is None:
                #             print("Stopping condition reached, no improvement in evaluation set")
                #             sys.exit(0)

                #     logging_loss = tr_loss


    dt = datetime.now() - t0
    print(f'[DBG] Duration: {dt}')
    return global_step, tr_loss / global_step

In [104]:
def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, evaluate=True)
    if args.local_rank in [-1, 0]:
        os.makedirs(eval_output_dir, exist_ok=True)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate
    )


    result = None


    return result

In [ ]:
global_step, tr_loss = train(args, train_dataset, model, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

In [ ]:
import transformers
import tokenizers

print(f'transformers.__version__: {transformers.__version__}')
print(f'torch.__version__: {torch.__version__}')
print(f'tokenizers.__version__: {tokenizers.__version__}')
print(f'wandb.__version__: {wandb.__version__}')